In [2]:
from dataclasses import dataclass
from typing import Literal, Optional
import numpy
import pandas

@dataclass
class Post:
    gallery: str
    title: str
    content: str
    date: numpy.datetime64
    keyword: str
    searchMode: Literal["accuracy", "latest"]
    disaster: Optional[bool]

In [19]:
from typing import Optional
import requests
from urllib.parse import quote
import bs4

@dataclass
class SearchResultPost:
    gallery: str
    title: str
    content: str
    date: numpy.datetime64

def search(query: str, page: int = 1, mode: Literal["latest", "accuracy"] = "accuracy") -> Optional[list[SearchResultPost]]:
    response = requests.get(f"https://search.dcinside.com/post/p/{page}/sort/{mode}/q/{quote(query, safe='.')}")
    if not (200 <= response.status_code < 299):
        print(response.text);
        return None
    
    list = []
    soup = bs4.BeautifulSoup(response.text)
    for element in soup.select(".sch_result_list>li"):
        gallery = element.select_one(".sub_txt").get_text()
        title = element.select_one(".tit_txt").get_text()
        content = element.select_one(".link_dsc_txt").get_text()
        date = pandas.to_datetime(element.select_one(".date_time").get_text())
        list.append(SearchResultPost(gallery, title, content, date))
    return list

In [20]:
from time import sleep

keywords = ["지진", "화재", "불남", "싱크홀", "근데"]

def scrapeKeyword(keyword: str, mode = Literal["accuracy", "latest"], pages: int = 5):
    labeledPosts = []
    for page in range(1, pages + 1):
        print(f"Keyword {keyword}, page {page}, mode {mode}")
        posts = search(keyword, page, mode)
        sleep(1)
        if posts == None:
            raise Exception(f"Failed to fetch search results for keyword {keyword}, page {page}")

        for post in posts:
            # print(f"{post.title} - {post.gallery} {post.date:%Y-%m-%d}\n{post.content}")
            labeledPosts.append(Post(gallery=post.gallery, title=post.title, content=post.content, date=post.date, keyword=keyword, disaster=None, searchMode=mode))
    return labeledPosts

posts = []

for keyword in keywords:
    posts += scrapeKeyword(keyword, mode="accuracy");
    posts += scrapeKeyword(keyword, mode="latest");

Keyword 지진, page 1, mode accuracy
Keyword 지진, page 2, mode accuracy
Keyword 지진, page 3, mode accuracy
Keyword 지진, page 4, mode accuracy
Keyword 지진, page 5, mode accuracy
Keyword 지진, page 1, mode latest
Keyword 지진, page 2, mode latest
Keyword 지진, page 3, mode latest
Keyword 지진, page 4, mode latest
Keyword 지진, page 5, mode latest
Keyword 화재, page 1, mode accuracy
Keyword 화재, page 2, mode accuracy
Keyword 화재, page 3, mode accuracy
Keyword 화재, page 4, mode accuracy
Keyword 화재, page 5, mode accuracy
Keyword 화재, page 1, mode latest
Keyword 화재, page 2, mode latest
Keyword 화재, page 3, mode latest
Keyword 화재, page 4, mode latest
Keyword 화재, page 5, mode latest
Keyword 불남, page 1, mode accuracy
Keyword 불남, page 2, mode accuracy
Keyword 불남, page 3, mode accuracy
Keyword 불남, page 4, mode accuracy
Keyword 불남, page 5, mode accuracy
Keyword 불남, page 1, mode latest
Keyword 불남, page 2, mode latest
Keyword 불남, page 3, mode latest
Keyword 불남, page 4, mode latest
Keyword 불남, page 5, mode latest
Keyword 싱크

In [21]:
df = pandas.DataFrame(posts)

In [23]:
df

,gallery,title,content,date,keyword,searchMode,disaster
0,지진,2024년 12월 6일 미국지진 분석,CA 깊이0.6km Ⅷ 쓰나미경보 ChatGPT의 말: 2024년 12월 5일 캘리...,2024-12-06 04:48:00,지진,accuracy,None
1,지진,2024년 11월 5일~2024년 12월 5일 30일간 미국지진 분석,↓2024-11-05~2024-12-05 미국지진 M3.7이상 ★ 오타로 인해 자료...,2024-12-05 16:21:00,지진,accuracy,None
2,러시아-우크라이나,"쿠바, 허리케인 끝나자 이번엔 규모 6.8 지진…""이런 지진은 처음""",지진 후 15회 이상 여진도 발생…쓰나미 위험은 없어 사망자 아직 보고되지 않았지만...,2024-11-11 08:37:00,지진,accuracy,None
3,일본 지진,11월17일~12월 5일 일본지진 Chatgpt분석,★ 오타로 인해 자료에 오류가 있을 가능성 있습니다. 2024-11-17 21:16...,2024-12-05 15:46:00,지진,accuracy,None
4,지진,11월17일~12월 5일 일본지진 Chatgpt분석,★ 오타로 인해 자료에 오류가 있을 가능성 있습니다. 2024-11-17 21:16...,2024-12-05 15:44:00,지진,accuracy,None
...,...,...,...,...,...,...,...
1242,메르스,띠닌 근데 애미가 말 무례하게 하면,발로 배 걷어차고 뺨 ㅈㄴ 때림 이년도 그냥 애비랑 똑같은 한남(후천적 남근)이라 ...,2024-12-09 15:34:00,근데,latest,None
1243,림월드,근데 시나리오 추가된 건축물에는 왜 하필 일반 전선이 기본적으로 들어감?,SOS도 엔진에 일반 전선 붙어있어서 거기서 전기 터진 적도 있고 지금 데드맨 스위...,2024-12-09 15:34:00,근데,latest,None
1244,원신 project,근데 차스카 분산 때문에 의외로 딜 박힘,마지막에 발사되는게 첫발인 바람탄이라 확산이 확정적으로 일어나서 적들 한군데 몰려있...,2024-12-09 15:34:00,근데,latest,None
1245,걸스플래닛999,근데에스파 왤케 인기 많노,음악 걍 그런데 베몬이나 있지가 더 괜찮은디 - dc official App,2024-12-09 15:34:00,근데,latest,None


In [24]:
df.to_csv("./data_dcinside/raw_data.csv")